In [2]:
import pandas as pd
import plotly
import plotly.express as px
import plotly.io as pio
import dash 

popTotal = pd.read_csv('Kidsdata-child-population-age-gender-Child-Population--by-A.csv')

cleaning = popTotal['Data'].str.replace('"', '')   # remove the "" and , 
cleaned = cleaning.str.replace(",","")
pData = cleaned.astype(int)

popTotal['Data'] = pData

popTotal['TimeFrame'] = (popTotal['TimeFrame']).astype(int)

popTotal = popTotal.loc[(popTotal['LocationType'] == 'County') & 
                        (popTotal['Gender'] == 'Total')]     

popTotal = popTotal.loc[(popTotal['AgeGroup']== 'Ages 0_2') |       # Only keep child population
                        (popTotal['AgeGroup']== 'Ages 3_5') |       # Younger than 10
                        (popTotal['AgeGroup']== 'Ages 6_10')]

popTotal = popTotal.loc[(popTotal['TimeFrame'] == 2000) | (popTotal['TimeFrame'] == 2002) |
                        (popTotal['TimeFrame'] == 2004) | (popTotal['TimeFrame'] == 2006) |
                        (popTotal['TimeFrame'] == 2008) | (popTotal['TimeFrame'] == 2010) |
                        (popTotal['TimeFrame'] == 2012) | (popTotal['TimeFrame'] == 2014) |
                        (popTotal['TimeFrame'] == 2017) | (popTotal['TimeFrame'] == 2019)]

popTotal = popTotal.groupby(['Location','TimeFrame'], as_index = False).sum()

popTotal = popTotal.drop(['FIPSCode'], axis = 1)

           Location  TimeFrame    Data
0    Alameda County       2000  221379
1    Alameda County       2002  222022
2    Alameda County       2004  215336
3    Alameda County       2006  209910
4    Alameda County       2008  210516
..              ...        ...     ...
575     Yuba County       2010   13203
576     Yuba County       2012   13397
577     Yuba County       2014   13328
578     Yuba County       2017   13143
579     Yuba County       2019   13037

[580 rows x 3 columns]


In [8]:
ccSpaces = pd.read_csv('Kidsdata-child-care-spaces-Child-Care-Spaces-in-Licensed-Fa.csv')

cleaning = ccSpaces["Data"].str.replace('"', '')   # remove the "" and , 
cleaned = cleaning.str.replace(",","")
ccData = cleaned.astype(int)

ccSpaces['Data'] = ccData

ccSpaces['TimeFrame'] = (ccSpaces['TimeFrame']).astype(int)

ccSpaces = ccSpaces.loc[(ccSpaces['LocationType'] == 'County') &
                        (ccSpaces['TypeofFacility'] == 'Total for Both Facility Types')]

ccSpaces = ccSpaces.drop(['FIPSCode'], axis = 1)

ccSpaces = ccSpaces.groupby(['Location','TimeFrame'], as_index = False).sum()

ccSpaces.rename(columns = {'Data':'ChildCareSpaces'}, inplace = True)

df = ccSpaces.assign(ChildPopulation = (popTotal['Data']))       #Combining data sets

print(df)

           Location  TimeFrame  ChildCareSpaces  ChildPopulation
0    Alameda County       2000            53538           221379
1    Alameda County       2002            53930           222022
2    Alameda County       2004            53859           215336
3    Alameda County       2006            52883           209910
4    Alameda County       2008            53227           210516
..              ...        ...              ...              ...
575     Yuba County       2010             2160            13203
576     Yuba County       2012             2064            13397
577     Yuba County       2014             2027            13328
578     Yuba County       2017             1341            13143
579     Yuba County       2019             1849            13037

[580 rows x 4 columns]


In [7]:
mask = df["Location"] == "Alameda County"

fig2 = px.bar(df, x="TimeFrame", y=["ChildCareSpaces","ChildPopulation"], title='Child Care Availability per County', barmode= 'group')
fig2.show()

In [9]:
from dash import Dash, dcc, html, Input, Output

counties = df['Location']

app = dash.Dash(__name__)


app.layout = html.Div([
    html.H4('Child Care Availability per County'),
    dcc.Dropdown(
        id="dropdown",
        options= counties,
        value='Alameda County',
        clearable=False,
    ),
    dcc.Graph(id="graph"),
])


@app.callback(
    Output("graph", "figure"), 
    Input("dropdown", "value"))
def update_bar_chart(county):
    
    mask = df["Location"] == county
    fig = px.bar(df[mask], x="TimeFrame", y=["ChildCareSpaces","ChildPopulation"], 
                 barmode="group")
    return fig


app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


Traceback (most recent call last):
  File "/Users/school/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/school/opt/anaconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 845, in launch_instance
    app.initialize(argv)
  File "/Users/school/opt/anaconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 88, in inner
    return method(app, *args, **kwargs)
  File "/Users/school/opt/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 632, in initialize
    self.init_sockets()
  File "/Users/school/opt/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 282, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  File "/Users/school/opt/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 229, in _bind_socket
    return self._try_bind_socket(s, port)
  File "/Users/school/opt/anacond

SystemExit: 1

/Users/school/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3452: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.

